Calculate Tractor Trips and movements

- Get a list of all direction ideintfied images (mike and Neil)
- function
    - find the first unclaimed image
    - if mike to > find neil to within time period > give both same trip id > if cant mark as unclaimed
    - if mike from find neil from in time period and give same trip id > if cant mark as unclaimed
    
    - if neil to find mike to in time period > if not mark as unclaimed
    - if neil from find mike from in time period > if not mark as unclaimed
    
    - Result - all jined up trip
        - combine trip tin movements > math to and from trip in righ ttime period

    - unclaimed
        - match unclaimed neils as not via mike
        - should be many unclaimed mike      



In [49]:
#cameras
MIKE = 1
NEIL = 2

#direction
TO = 1
FROM = 0

#status
NOT_CLAIMED = 0
CLAIMED = 1
ORPHAN = 2

In [50]:
import pandas as pd
import psycopg2
import db_config as creds
import os
import time
import datetime
from datetime import datetime  
from datetime import timedelta  

from PIL import Image

In [51]:
class tractor:
  def __init__(self, filename, datetaken, cameraid,cameraname,direction):
    self.datetaken = datetaken
    self.filename = filename
    self.cameraname = cameraname
    self.cameraid = cameraid
    self.direction = direction
        
   

## Database Functions

In [52]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    #print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    #print ("disconnected!")

In [53]:
def getTractors():

    # Connecting to DB
    conn, cursor = connect()
    
    #get list of all tractor images with a direction
    dat = pd.read_sql(('select filename, datetaken_adjusted, cameraid, cameraname, true_direction from images '
                     'where datetaken BETWEEN %(dstart)s AND %(dfinish)s and true_direction is not null order by datetaken_adjusted asc'),
                   conn,params={"dstart":datetime(2019,8,8,0,0),"dfinish":datetime(2020,7,27,0,0)})

       
    dat["claimed"] = NOT_CLAIMED
    dat["tripid"] = 0
    #dat["movementid"] = 0
    
    # Disconnect from DB
    disconnect(conn, cursor)
    
    print(dat)
    return dat

def findTrips(tractors):

    unclaimed = len(tractors[tractors['claimed'] == 0])
                    
    while (unclaimed > 0):
    
        #print(unclaimed) 
    
        #find next unclaimed tractor
        next_t = nextUnclaimed(tractors)
   
        #look for a related on either mike or neil           
        related_t = lookForRelated(next_t, tractors)
    
        #Get the next trip id
        tripid = tractors["tripid"].max() + 1
    
        #orphan, we cant find a matching trip image, set trip id, set status      
        if (next_t.filename == related_t.filename):
            
            print ("orphaned")
            tractors.loc[tractors.filename == next_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
        
        else:
        
            print ("matched") 
            
            #update image
            tractors.loc[tractors.filename == next_t.filename, 'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
        
            #update related image  
            tractors.loc[tractors.filename == related_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == related_t.filename,'tripid'] = tripid 
    
            print("Original",next_t.filename,"Related to",related_t.filename)
        
        unclaimed = len(tractors[tractors['claimed'] == 0])
    else:
        print(tractors.head())
        print("trips mapped")    
  

def nextUnclaimed(tractors):

    #find the next tractor in the dataframe that is unclames
    t = tractors[tractors.claimed == 0].iloc[0]
    nextTractor = tractor(t[0],t[1],t[2],t[3],t[4])
   
    return nextTractor

def lookForRelated(t, tractors):  

    #print (t.filename)
    time_min, time_max = getTimeWindow(t)
    
    #mike
    if (t.cameraid == MIKE):
        
        #print ("mike image-",t.filename)
        # look for related neil to image 
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0) 
                             & (tractors.cameraid == NEIL)
                             & (tractors.true_direction == t.direction)
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
        
        except:
            #print("No matching Neil image")
            return t #just return the original
        
    else:    
        #print ("neil image-",t.filename)
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0)
                             & (tractors.cameraid == MIKE)   
                             & (tractors.true_direction == t.direction) 
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
        
        except:
            #print("No matching mike image")
            return t #just return the original
        
    return related_t
    
   
def getTimeWindow(t):
    
    #return min and max time range
    # it takes between 4 and 8 minutes to get between mike and neil 
    mins_min, mins_max = 4, 13
    
    #if mike to, then add time for neil
    if (t.cameraid == MIKE and t.direction == TO):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max)
    
    elif (t.cameraid == MIKE and t.direction == FROM):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == TO):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == FROM):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max) 
    
    
    #print("taken-",t.datetaken,"min-",minTime,"max-",maxTime)
    
    return minTime, maxTime    
    
    
def updateDb(tractors):
    
    # Connecting to DB
    conn, cursor = connect()
   
    sql = """UPDATE images SET trip_id_all = %s where filename = %s"""
  
    for index, row in tractors.iterrows():
        filename = row["filename"]
        tripid = row["tripid"] 
  
        cursor.execute(sql, (tripid,filename))
        conn.commit()
    
    print ("database updated")
    # Disconnect from DB
    disconnect(conn, cursor)
    

In [54]:
#get list of all tractors
df_tractors = getTractors()

#find trips
findTrips(df_tractors)
print(df_tractors)

#update database
updateDb(df_tractors)


          filename  datetaken_adjusted  cameraid cameraname  true_direction  \
0     01PT0072.JPG 2019-08-13 09:56:48         2       Neil               1   
1     01PT0073.JPG 2019-08-13 10:10:16         2       Neil               1   
2     01PT0079.JPG 2019-08-13 11:03:06         2       Neil               0   
3     01PT0110.JPG 2019-08-14 08:24:00         2       Neil               1   
4     01PT0114.JPG 2019-08-14 14:43:56         2       Neil               0   
...            ...                 ...       ...        ...             ...   
4071  08Mk5225.JPG 2020-07-24 12:10:14         1       Mike               1   
4072  08Mk5287.JPG 2020-07-24 13:26:06         1       Mike               1   
4073  08Mk5336.JPG 2020-07-24 14:11:52         1       Mike               0   
4074  08Mk5439.JPG 2020-07-24 15:23:02         1       Mike               1   
4075  08Mk5685.JPG 2020-07-24 21:14:50         1       Mike               0   

      claimed  tripid  
0           0       0  
1  

orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
o

matched
Original 13zPT196.JPG Related to 01Mk2381.JPG
matched
Original 01Mk2411.JPG Related to 13zPT198.JPG
matched
Original 13zPT199.JPG Related to 01Mk2430.JPG
orphaned
orphaned
matched
Original 01Mk2439.JPG Related to 13zPT202.JPG
matched
Original 13zPT203.JPG Related to 01Mk2446.JPG
orphaned
matched
Original 13zPT209.JPG Related to 01Mk2492.JPG
matched
Original 01Mk2504.JPG Related to 13zPT213.JPG
matched
Original 13zPT214.JPG Related to 01Mk2511.JPG
matched
Original 01Mk2527.JPG Related to 13zPT216.JPG
matched
Original 13zPT217.JPG Related to 01Mk2551.JPG
matched
Original 01Mk2562.JPG Related to 13zPT218.JPG
matched
Original 13zPT220.JPG Related to 01Mk2580.JPG
orphaned
matched
Original 01Mk2589.JPG Related to 13zPT222.JPG
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
o

matched
Original 14PT1262.JPG Related to 04Mk0231.JPG
matched
Original 04Mk0220.JPG Related to 14PT1268.JPG
matched
Original 14PT1265.JPG Related to 04Mk0243.JPG
matched
Original 14PT1266.JPG Related to 04Mk0245.JPG
orphaned
matched
Original 14PT1269.JPG Related to 04Mk0261.JPG
matched
Original 04Mk0255.JPG Related to 14PT1270.JPG
matched
Original 04Mk0264.JPG Related to 14PT1271.JPG
matched
Original 04Mk0269.JPG Related to 14PT1275.JPG
matched
Original 14PT1272.JPG Related to 04Mk0273.JPG
matched
Original 14PT1273.JPG Related to 04Mk0282.JPG
matched
Original 04Mk0277.JPG Related to 14PT1276.JPG
matched
Original 04Mk0287.JPG Related to 14PT1280.JPG
matched
Original 04Mk0291.JPG Related to 14PT1281.JPG
matched
Original 14PT1278.JPG Related to 04Mk0308.JPG
matched
Original 14PT1279.JPG Related to 04Mk0311.JPG
matched
Original 04Mk0298.JPG Related to 14PT1286.JPG
matched
Original 14PT1282.JPG Related to 04Mk0315.JPG
matched
Original 14PT1283.JPG Related to 04Mk0316.JPG
orphaned
orphaned
m

Original 04Mk2833.JPG Related to 15PT1525.JPG
orphaned
matched
Original 15PT1526.JPG Related to 04Mk2855.JPG
orphaned
matched
Original 04Mk2850.JPG Related to 15PT1528.JPG
matched
Original 04Mk2868.JPG Related to 15PT1530.JPG
matched
Original 15PT1529.JPG Related to 04Mk2901.JPG
matched
Original 04Mk2886.JPG Related to 15PT1532.JPG
orphaned
orphaned
matched
Original 04Mk2898.JPG Related to 15PT1534.JPG
matched
Original 15PT1533.JPG Related to 04Mk2906.JPG
matched
Original 04Mk2903.JPG Related to 15PT1535.JPG
matched
Original 04Mk2917.JPG Related to 15PT1538.JPG
matched
Original 15PT1536.JPG Related to 04Mk2926.JPG
matched
Original 15PT1537.JPG Related to 04Mk2929.JPG
matched
Original 04Mk2921.JPG Related to 15PT1539.JPG
matched
Original 15PT1540.JPG Related to 04Mk2953.JPG
matched
Original 15PT1541.JPG Related to 04Mk2960.JPG
matched
Original 04Mk2946.JPG Related to 15PT1542.JPG
matched
Original 04Mk2969.JPG Related to 15PT1544.JPG
matched
Original 15PT1543.JPG Related to 04Mk2982.JPG


matched
Original 04Mk3737.JPG Related to 15PT1689.JPG
matched
Original 04Mk3751.JPG Related to 15PT1692.JPG
matched
Original 15PT1690.JPG Related to 04Mk3760.JPG
matched
Original 04Mk3755.JPG Related to 15PT1693.JPG
matched
Original 15PT1691.JPG Related to 04Mk3763.JPG
matched
Original 15PT1694.JPG Related to 04Mk3767.JPG
orphaned
matched
Original 04Mk3775.JPG Related to 15PT1696.JPG
matched
Original 04Mk3779.JPG Related to 15PT1698.JPG
matched
Original 15PT1697.JPG Related to 04Mk3788.JPG
matched
Original 04Mk3784.JPG Related to 15PT1700.JPG
matched
Original 15PT1699.JPG Related to 04Mk3792.JPG
matched
Original 04Mk3796.JPG Related to 15PT1702.JPG
orphaned
orphaned
matched
Original 04Mk3805.JPG Related to 15PT1704.JPG
matched
Original 04Mk3810.JPG Related to 15PT1705.JPG
orphaned
matched
Original 04Mk3814.JPG Related to 15PT1706.JPG
matched
Original 15PT1707.JPG Related to 04Mk3826.JPG
orphaned
matched
Original 04Mk3829.JPG Related to 15PT1710.JPG
matched
Original 04Mk3834.JPG Related

orphaned
matched
Original 06Mk0708.JPG Related to 20PT1568.JPG
matched
Original 20PT1569.JPG Related to 06Mk0724.JPG
matched
Original 20PT1570.JPG Related to 06Mk0732.JPG
orphaned
orphaned
matched
Original 20PT1572.JPG Related to 06Mk0756.JPG
matched
Original 06Mk0760.JPG Related to 20PT1574.JPG
orphaned
matched
Original 06Mk0780.JPG Related to 20PT1577.JPG
orphaned
matched
Original 06Mk0794.JPG Related to 20PT1581.JPG
orphaned
orphaned
matched
Original 06Mk0804.JPG Related to 20PT1582.JPG
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
o

orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
orphaned
       filename  datetaken_adjusted  cameraid cameraname  true_direction  \
0  01PT0072.JPG 2019-08-13 09:56:48         2       Neil               1   
1  01PT0073.JPG 2019-08-13 10:10:16   